<a href="https://colab.research.google.com/github/mitsouma/IOT_Anomalies_Detection_System_Based_on_Tensor_Decomposition/blob/main/Tensorization_of_iot_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import math
import h5py
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from typing import List, Optional, Union, Dict, Tuple
from scipy.sparse import dok_matrix, coo_matrix
import networkx as nx

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


                         **Tensorization of the CIC_IoT2024 Dataset**

This work focuses on the tensorization of the CIC_IoT2024 dataset, which was collected at the Canadian Institute for Cybersecurity (CIC). The dataset is specifically designed for two major tasks: IoT device identification and anomaly detection.

It consists of two sets of features:

Behavior-based features (such as HTTPS handshake information, User-Agent strings)

Flow-based features (including stream, channel, and jitter metrics over time intervals)

The dataset includes network traffic from 105 IoT devices and simulates 33 types of attacks, grouped into seven categories: DDoS, DoS, Reconnaissance, Web-based, Brute Force, Spoofing, and Mirai. All attacks are launched by malicious IoT devices targeting other IoT devices.

Tensorization allows for the transformation of this multi-source data into structured multi-dimensional arrays (tensors), capturing latent patterns and relationships across various dimensions (e.g., time, device, feature type). This approach supports more effective anomaly detection and classification in complex IoT environments.

We will construct two different tensors

1. Behavior based features tensor
2. Flow based tensor
The two tensors will be used for anomalies detection. For all class of features, we have taken one benign csv file and three anomalous for different attack in wich have taken a small proportion


**Let construct the Behavior based features tensor**


In [46]:
df_b_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic.csv', on_bad_lines='skip', engine='python') #skip bad lines
df_b_2=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic1.csv') #skip bad lines
df_DDoS=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/anomalies_packetbased/DDoS-TCP_Flood.csv') #skip bad lines
df_Mirai=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/anomalies_packetbased/Mirai-greip_flood.csv') #skip bad lines

<ipython-input-46-225bd171e4f4>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_DDoS=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/anomalies_packetbased/DDoS-TCP_Flood.csv') #skip bad lines
<ipython-input-46-225bd171e4f4>:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Mirai=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/anomalies_packetbased/Mirai-greip_flood.csv') #skip bad lines


In [16]:
df_b_1.head()

,stream,src_mac,dst_mac,src_ip,dst_ip,src_port,dst_port,inter_arrival_time,time_since_previously_displayed_frame,port_class_dst,...,sum_p,min_p,max_p,med_p,average_p,var_p,q3_p,q1_p,iqr_p,l3_ip_dst_count
0,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891,443,0.000000,0.000000,1,...,58160.0,2908.0,2908.0,2908.0,2908.0,0.000000,2908.0,2908.0,0.0,1.0
1,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891,443,0.000164,0.000164,1,...,58160.0,2908.0,2908.0,2908.0,2908.0,0.000000,2908.0,2908.0,0.0,1.0
2,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891,443,0.000269,0.000105,1,...,56712.0,1460.0,2908.0,2908.0,2835.6,104835.200000,2908.0,2908.0,0.0,1.0
3,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891,443,0.000414,0.000145,1,...,55264.0,1460.0,2908.0,2908.0,2763.2,198635.115789,2908.0,2908.0,0.0,1.0
4,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891,443,0.001800,0.001386,1,...,53816.0,1460.0,2908.0,2908.0,2690.8,281399.747368,2908.0,2908.0,0.0,1.0


In [47]:
df_b_1['label']=0
df_b_2['label']=0
df_DDoS['label']=1
df_Mirai['label']=2

In [49]:
# Shape of each dataset
print("Shape of df_b_1:", df_b_1.shape)
print("Shape of df_b_2:", df_b_2.shape)
print("Shape of df_DDoS:", df_DDoS.shape)
print("Shape of df_Mirai:", df_Mirai.shape)

Shape of df_b_1: (142862, 136)
Shape of df_b_2: (128853, 136)
Shape of df_DDoS: (85423, 136)
Shape of df_Mirai: (213030, 136)


In [50]:
# Infos of each dataset
print('for ben_1','*'*20,df_b_1.info())
print(''*40)
print('for ben_2','*'*20,df_b_2.info())
print(''*40)
print(''*40)
print('for DDoS','*'*20,df_DDoS.info())
print(''*40)
print('for Mirai','*'*20,df_Mirai.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142862 entries, 0 to 142861
Columns: 136 entries, stream to label
dtypes: float64(98), int64(22), object(16)
memory usage: 148.2+ MB
for ben_1 ******************** None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128853 entries, 0 to 128852
Columns: 136 entries, stream to label
dtypes: float64(118), int64(2), object(16)
memory usage: 133.7+ MB
for ben_2 ******************** None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85423 entries, 0 to 85422
Columns: 136 entries, stream to label
dtypes: float64(97), int64(23), object(16)
memory usage: 88.6+ MB
for DDoS ******************** None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213030 entries, 0 to 213029
Columns: 136 entries, stream to label
dtypes: float64(97), int64(23), object(16)
memory usage: 221.0+ MB
for Mirai ******************** None


In [51]:
#we drop the na values in src_ip,dst_ip
df_b_1.dropna(subset=['src_ip','dst_ip'],inplace=True)
df_b_2.dropna(subset=['src_ip','dst_ip'],inplace=True)
df_DDoS.dropna(subset=['src_ip','dst_ip'],inplace=True)
df_Mirai.dropna(subset=['src_ip','dst_ip'],inplace=True)

In [52]:
#we drop the duplicated samples
df_b_1.drop_duplicates(inplace=True)
df_b_2.drop_duplicates(inplace=True)
df_DDoS.drop_duplicates(inplace=True)
df_Mirai.drop_duplicates(inplace=True)

In [111]:
df_Mirai['src_ip'].value_counts().head(30)

,count
src_ip,
192.168.137.189,44225
192.168.137.131,40478
192.168.137.227,31227
192.168.137.177,30713
192.168.137.220,28816
192.168.137.81,12991
192.168.137.242,12733
192.168.137.123,787
192.168.137.138,621


In [113]:
L_DDoS=['192.168.137.123',
       '157.249.81.141',
'192.168.137.224',
'255.255.255.255',
'192.168.137.1',
       ]
L_Mirai=['192.168.137.242', # The indent here was incorrect
'192.168.137.123',
'192.168.137.138',
'54.230.48.19',
'192.168.137.41']

In [114]:
#We select some dst_ip in D and DDoS and some Src_ip in Mirai datasets to corrupt the benign
df_DDoS[df_DDoS['dst_ip'].isin(L_DDoS)]['src_ip'].value_counts()

,count
src_ip,
192.168.137.17,23837
192.168.137.195,23389
192.168.137.134,17186
192.168.137.156,13121
192.168.137.15,5485
192.168.137.109,1665
192.168.137.224,112
13.225.210.183,56
157.249.81.141,55


In [116]:
df_Mirai[df_Mirai['src_ip'].isin(L_Mirai)]['dst_ip'].shape

(15132,)

Each stream represents a different kind of communication between two IOT devices, so we just consider the stream which are common

In [144]:
df=pd.concat([df_b_1,df_b_2,df_DDoS[df_DDoS['dst_ip'].isin(L_DDoS)],df_Mirai[df_Mirai['src_ip'].isin(L_Mirai)]],axis=0)

In [145]:
  df.shape

(356967, 136)

In [146]:
df.describe()

,stream,src_port,dst_port,inter_arrival_time,time_since_previously_displayed_frame,port_class_dst,l4_tcp,l4_udp,ttl,eth_size,...,min_p,max_p,med_p,average_p,var_p,q3_p,q1_p,iqr_p,l3_ip_dst_count,label
count,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,356967.000000,...,356966.000000,356966.000000,356966.000000,356966.000000,3.569660e+05,356966.000000,356966.000000,356966.000000,356966.000000,356967.000000
mean,31731.890082,27958.052921,19318.972180,618.166012,0.009287,1.871439,0.768396,0.215082,96.922155,345.967563,...,57.173145,1063.044060,221.180932,290.835941,3.247544e+05,356.746503,121.428979,235.317525,14.016738,0.322856
std,65424.687820,22380.564928,22280.789322,658.014443,0.019591,0.712457,0.421858,0.410879,68.922093,739.619610,...,228.842944,1929.487166,473.042213,488.756972,1.065605e+06,630.722476,349.737559,511.214674,17.581483,0.550819
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,50.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000
25%,55.000000,443.000000,443.000000,15.348582,0.000003,1.000000,1.000000,0.000000,64.000000,60.000000,...,6.000000,12.000000,6.000000,12.000000,0.000000e+00,12.000000,6.000000,0.000000,3.000000,0.000000
50%,457.000000,33255.000000,4070.000000,420.725526,0.000929,2.000000,1.000000,0.000000,64.000000,66.000000,...,12.000000,210.000000,18.000000,57.800000,4.127566e+03,48.750000,12.000000,16.000000,8.000000,0.000000
75%,11267.500000,49154.000000,41487.000000,1010.355973,0.009622,2.000000,1.000000,0.000000,64.000000,206.000000,...,12.000000,1310.000000,65.000000,313.500000,9.523094e+04,204.250000,12.000000,96.000000,17.000000,1.000000
max,264076.000000,65533.000000,65535.000000,2402.628016,0.484137,3.000000,1.000000,1.000000,255.000000,30474.000000,...,4356.000000,30420.000000,4356.000000,4356.000000,4.387091e+07,5804.000000,4356.000000,4374.000000,93.000000,2.000000


In [143]:
#Data Description
df.head(20)

,stream,src_mac,dst_mac,src_ip,dst_ip,src_port,dst_port,inter_arrival_time,time_since_previously_displayed_frame,port_class_dst,...,min_p,max_p,med_p,average_p,var_p,q3_p,q1_p,iqr_p,l3_ip_dst_count,label
0,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000000,0.000000,1.0,...,2908.0,2908.0,2908.0,2908.00,0.000000e+00,2908.0,2908.0,0.0,1.0,0
1,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000164,0.000164,1.0,...,2908.0,2908.0,2908.0,2908.00,0.000000e+00,2908.0,2908.0,0.0,1.0,0
2,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000269,0.000105,1.0,...,1460.0,2908.0,2908.0,2835.60,1.048352e+05,2908.0,2908.0,0.0,1.0,0
3,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000414,0.000145,1.0,...,1460.0,2908.0,2908.0,2763.20,1.986351e+05,2908.0,2908.0,0.0,1.0,0
4,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.001800,0.001386,1.0,...,1460.0,2908.0,2908.0,2690.80,2.813997e+05,2908.0,2908.0,0.0,1.0,0
5,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.001961,0.000161,1.0,...,1460.0,4356.0,2908.0,2763.20,4.193408e+05,2908.0,2908.0,0.0,1.0,0
6,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.002129,0.000168,1.0,...,1460.0,4356.0,2908.0,2690.80,5.021054e+05,2908.0,2908.0,0.0,1.0,0
7,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008073,0.005944,1.0,...,497.0,4356.0,2908.0,2570.25,7.376284e+05,2908.0,2546.0,362.0,1.0,0
8,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008075,0.000002,1.0,...,497.0,4356.0,2908.0,2497.85,7.909834e+05,2908.0,1460.0,1448.0,1.0,0
9,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008173,0.000098,1.0,...,497.0,4356.0,2908.0,2497.85,7.909834e+05,2908.0,1460.0,1448.0,1.0,0


In [147]:
df.drop_duplicates(inplace=True)

In [148]:
data=df[['src_ip','dst_ip','sum_p','inter_arrival_time']].dropna()

In [149]:
data.shape

(356966, 4)

**First suggestion of the tensor**

In [ ]:
# Tensor=[Src IP, Dst IP, features] where features:{count,time_inter_arrival,sum_p}


In [150]:
def create_tensor(df):
    # Extract unique source and destination IPs
    src_ips = df['src_ip'].unique()
    dst_ips = df['dst_ip'].unique()

    # Create mappings for src_ip and dst_ip to indices
    src_ip_to_idx = {ip: idx for idx, ip in enumerate(src_ips)}
    dst_ip_to_idx = {ip: idx for idx, ip in enumerate(dst_ips)}

    # Initialize adjacency tensor [Src IP, Dst IP, features]
    tensor = np.zeros((len(src_ips), len(dst_ips), 3))  # 3 features (count, bytes, duration)

    # Group by source and destination IPs
    grouped = df.groupby(['src_ip', 'dst_ip'])

    for (src_ip, dst_ip), group in grouped:
        src_idx = src_ip_to_idx[src_ip]
        dst_idx = dst_ip_to_idx[dst_ip]

        # Aggregated features
        tensor[src_idx, dst_idx, 0] = len(group)  # count
        tensor[src_idx, dst_idx, 1] = group['sum_p'].sum()  # bytes (sum over group)
        tensor[src_idx, dst_idx, 2] = group['inter_arrival_time'].mean()  # duration (average)

    return tensor, src_ip_to_idx, dst_ip_to_idx

In [151]:
tensor, src_ip_to_idx, dst_ip_to_idx=create_tensor(data)

In [152]:
tensor.shape

(427, 429, 3)

In [153]:
tensor.size

549549

In [154]:
np.count_nonzero(tensor)

3207

Second Proposition
In this proposition, the tensor will be constructed using 4 modes: n_sample x (stream,chanel,jitter,src_ip,src_ip_mac) x (count,var,mean) x time

In [156]:
L=['stream_1_count',
'stream_1_mean',
'stream_1_var',
'src_ip_1_count',
'src_ip_1_mean',
'src_ip_1_var',
'src_ip_mac_1_count',
'src_ip_mac_1_mean',
'src_ip_mac_1_var',
'channel_1_count',
'channel_1_mean',
'channel_1_var',
'stream_jitter_1_sum',
'stream_jitter_1_mean',
'stream_jitter_1_var',
'stream_5_count',
'stream_5_mean',
'stream_5_var',
'src_ip_5_count',
'src_ip_5_mean',
'src_ip_5_var',
'src_ip_mac_5_count',
'src_ip_mac_5_mean',
'src_ip_mac_5_var',
'channel_5_count',
'channel_5_mean',
'channel_5_var',
'stream_jitter_5_sum',
'stream_jitter_5_mean',
'stream_jitter_5_var',
'stream_10_count',
'stream_10_mean',
'stream_10_var',
'src_ip_10_count',
'src_ip_10_mean',
'src_ip_10_var',
'src_ip_mac_10_count',
'src_ip_mac_10_mean',
'src_ip_mac_10_var',
'channel_10_count',
'channel_10_mean',
'channel_10_var',
'stream_jitter_10_sum',
'stream_jitter_10_mean',
'stream_jitter_10_var',
'stream_30_count',
'stream_30_mean',
'stream_30_var',
'src_ip_30_count',
'src_ip_30_mean',
'src_ip_30_var',
'src_ip_mac_30_count',
'src_ip_mac_30_mean',
'src_ip_mac_30_var',
'channel_30_count',
'channel_30_mean',
'channel_30_var',
'stream_jitter_30_sum',
'stream_jitter_30_mean',
'stream_jitter_30_var',
'stream_60_count',
'stream_60_mean',
'stream_60_var',
'src_ip_60_count',
'src_ip_60_mean',
'src_ip_60_var',
'src_ip_mac_60_count',
'src_ip_mac_60_mean',
'src_ip_mac_60_var',
'channel_60_count',
'channel_60_mean',
'channel_60_var',
'stream_jitter_60_sum',
'stream_jitter_60_mean',
'stream_jitter_60_var']

In [157]:
df_b_1.dropna(subset=L, inplace=True)
df_b_2.dropna(subset=L, inplace=True)
df_DDoS.dropna(subset=L, inplace=True)
df_Mirai.dropna(subset=L, inplace=True)

In [159]:
data_2=df[L]

In [160]:
data_2.shape

(356967, 75)

In [161]:
def create_4d_tensor(df):
    # Dimensions
    features = ['stream', 'src_ip', 'src_ip_mac', 'channel', 'stream_jitter']
    metrics = ['count', 'mean', 'var']
    time_windows = [1, 5, 10, 30, 60]

    n_samples = df.shape[0]
    tensor = np.zeros((n_samples, len(features), len(metrics), len(time_windows)))

    for i, feature in enumerate(features):
        for j, metric in enumerate(metrics):
            for k, t in enumerate(time_windows):
                # Correction : pour stream_jitter, remplacer "count" par "sum"
                metric_name = "sum" if feature == "stream_jitter" and metric == "count" else metric
                col = f"{feature}_{t}_{metric_name}"
                tensor[:, i, j, k] = df[col].values

    return tensor


In [163]:
tensor=create_4d_tensor(data_2)

In [164]:
tensor.shape

(356967, 5, 3, 5)

In [165]:
tensor.size

26772525

In [166]:
nonzero_elements = np.count_nonzero(tensor)
total_elements = tensor.size
sparsity_ratio = 1 - (nonzero_elements / total_elements)

In [168]:
nonzero_elements

24718357

**Construction of the Flow based tensor**

In [4]:
#Tensor construction for flowbased_features
df1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_flowbased/BenignTraffic.pcap_Flow.csv') #skip bad lines
df2=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_flowbased/BenignTraffic1.pcap_Flow.csv') #skip bad lines
df3=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_flowbased/BenignTraffic2.pcap_Flow.csv')
df4=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_flowbased/BenignTraffic3.pcap_Flow.csv')
df5=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Anomalies_fbased/DDoS-HTTP_Flood-.pcap_Flow.csv')
df6=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Anomalies_fbased/DoS-HTTP_Flood.pcap_Flow.csv') #skip bad lines
 #skip bad lines

In [5]:
df1['Timestamp']=pd.to_datetime(df1['Timestamp'])
df2['Timestamp']=pd.to_datetime(df2['Timestamp'])
df3['Timestamp']=pd.to_datetime(df3['Timestamp'])
df4['Timestamp']=pd.to_datetime(df4['Timestamp'])
df5['Timestamp']=pd.to_datetime(df5['Timestamp'])
df6['Timestamp']=pd.to_datetime(df6['Timestamp'])

<ipython-input-5-784bfd6b29cb>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['Timestamp']=pd.to_datetime(df1['Timestamp'])
<ipython-input-5-784bfd6b29cb>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Timestamp']=pd.to_datetime(df2['Timestamp'])
<ipython-input-5-784bfd6b29cb>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df4['Timestamp']=pd.to_datetime(df4['Timestamp'])
<ipython-input-5-784bfd6b29cb>:5: UserWarning: Parsing dates in %d/%m/%Y %I:%M:%S %p format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silen

In [6]:
df1['Jour']=df1['Timestamp'].dt.date
df2['Jour']=df2['Timestamp'].dt.date
df3['Jour']=df3['Timestamp'].dt.date
df4['Jour']=df4['Timestamp'].dt.date
df5['Jour']=df5['Timestamp'].dt.date
df6['Jour']=df6['Timestamp'].dt.date

In [7]:
df1['Jour']=pd.to_datetime(df1['Jour'])
df2['Jour']=pd.to_datetime(df2['Jour'])
df3['Jour']=pd.to_datetime(df3['Jour'])
df4['Jour']=pd.to_datetime(df4['Jour'])
df5['Jour']=pd.to_datetime(df5['Jour'])
df6['Jour']=pd.to_datetime(df6['Jour'])

In [8]:
# Création d'un sous-dataset pour le jour 2022-07-10 à partir de df1 et df2
df_ben_day_1 = pd.concat([
    df1[df1['Jour'] == '2022-07-10'],
    df2[df2['Jour'] == '2022-07-10']
], axis=0)

# Création d'un sous-dataset pour le jour 2022-08-10 à partir de df2, df3 et df4
df_ben_day_2 = pd.concat([
    df2[df2['Jour'] == '2022-08-10'],
    df3[df3['Jour'] == '2022-08-10'],
    df4[df4['Jour'] == '2022-08-10']
], axis=0)
df5_day_1 = df5[df5['Jour'] == '2022-09-14']
df5_day_2 = df5[df5['Jour'] == '2022-11-07']
df6_day_1 = df6[df6['Jour'] == '2022-08-08']
df6_day_2 = df6[df6['Jour'] == '2022-09-08']

In [9]:
df_ben_day_1.drop_duplicates(inplace=True)
df_ben_day_2.drop_duplicates(inplace=True)
df5_day_1.drop_duplicates(inplace=True)
df5_day_2.drop_duplicates(inplace=True)
df6_day_1.drop_duplicates(inplace=True)
df6_day_2.drop_duplicates(inplace=True)

<ipython-input-9-ba655dd153ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_1.drop_duplicates(inplace=True)
<ipython-input-9-ba655dd153ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_2.drop_duplicates(inplace=True)
<ipython-input-9-ba655dd153ea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6_day_1.drop_duplicates(inplace=True)
<ipython-input-9-ba655dd153ea>:6: SettingWithCopyWarning: 
A value is trying to

In [10]:
df_ben_day_1.drop_duplicates(inplace=True)
df_ben_day_2.drop_duplicates(inplace=True)
df5_day_1.drop_duplicates(inplace=True)
df5_day_2.drop_duplicates(inplace=True)
df6_day_1.drop_duplicates(inplace=True)
df6_day_2.drop_duplicates(inplace=True)

<ipython-input-10-ba655dd153ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_1.drop_duplicates(inplace=True)
<ipython-input-10-ba655dd153ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_2.drop_duplicates(inplace=True)
<ipython-input-10-ba655dd153ea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6_day_1.drop_duplicates(inplace=True)
<ipython-input-10-ba655dd153ea>:6: SettingWithCopyWarning: 
A value is tryin

In [11]:

# 2. Création de deux DataFrames : un pour la tranche 13h, un pour les autres
mask_13h = (df6_day_1['Timestamp'].dt.hour == 13)
df_13h = df6_day_1[mask_13h]
df_autres = df6_day_1[~mask_13h]

# 3. Identifier les Dst IP uniquement présentes dans df_13h
dst_ip_13h_uniques = set(df_13h['Dst IP']) - set(df_autres['Dst IP'])

# 4. Garder uniquement ces Dst IP dans le dataset de 13h
df_resultat = df_13h[df_13h['Dst IP'].isin(dst_ip_13h_uniques)]

# Optionnel : sauvegarder le résultat
# df_resultat.to_csv("dst_ip_uniques_13h.csv", index=False)


In [12]:
df_resultat['Dst IP'].value_counts()

,count
Dst IP,
192.168.137.131,52748
108.138.125.109,19
54.91.82.76,16
3.136.72.65,16
52.92.210.242,16
...,...
142.251.33.161,1
206.108.0.132,1
120.24.36.178,1


In [13]:
L_5_1=['192.168.137.41', '157.249.81.141', '109.95.152.166','192.168.137.4','192.168.137.89'
       ]
L_6_1=['192.168.137.131',
'108.138.125.109',
'54.91.82.76',
'3.136.72.65',
'52.92.210.242'
       ]

In [16]:
df_ben_day_1['Heure'] = df_ben_day_1['Timestamp'].dt.strftime('%H:%M:%S')
df_ben_day_2['Heure'] = df_ben_day_2['Timestamp'].dt.strftime('%H:%M:%S')
df5_day_1['Heure'] = df5_day_1['Timestamp'].dt.strftime('%H:%M:%S')
df6_day_1['Heure'] = df6_day_1['Timestamp'].dt.strftime('%H:%M:%S')

<ipython-input-16-321c517596cd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_1['Heure'] = df5_day_1['Timestamp'].dt.strftime('%H:%M:%S')
<ipython-input-16-321c517596cd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6_day_1['Heure'] = df6_day_1['Timestamp'].dt.strftime('%H:%M:%S')


In [17]:
df_ben_day_2['Heure']=pd.to_datetime(df_ben_day_2['Heure'])
df5_day_1['Heure']=pd.to_datetime(df5_day_1['Heure'])
df6_day_1['Heure']=pd.to_datetime(df6_day_1['Heure'])

<ipython-input-17-17d5e6f8557a>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ben_day_2['Heure']=pd.to_datetime(df_ben_day_2['Heure'])
<ipython-input-17-17d5e6f8557a>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df5_day_1['Heure']=pd.to_datetime(df5_day_1['Heure'])
<ipython-input-17-17d5e6f8557a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_1['Heure']=pd.to_datetime(df5_day_1['Heure'])
<ipython-input-17-17d5e6f8557a>:3: UserWarning: Could not infer f

In [18]:
df5_day_1[df5_day_1['Dst IP'].isin(L_5_1)]['Heure'].sort_values()

,Heure
0,2025-04-23 10:47:00
1,2025-04-23 10:47:00
343151,2025-04-23 10:47:00
38,2025-04-23 10:47:00
320036,2025-04-23 10:47:01
...,...
291756,2025-04-23 11:14:41
355728,2025-04-23 11:14:42
341325,2025-04-23 11:14:42
330321,2025-04-23 11:14:43


In [19]:
df6_day_1[df6_day_1['Dst IP'].isin(L_6_1)]['Heure'].sort_values()

,Heure
3,2025-04-23 13:25:03
5,2025-04-23 13:25:03
41823,2025-04-23 13:25:03
1025,2025-04-23 13:25:08
1026,2025-04-23 13:25:08
...,...
66044,2025-04-23 13:37:00
66043,2025-04-23 13:37:00
66042,2025-04-23 13:37:00
66053,2025-04-23 13:37:00


In [20]:
df_ben_day_2['relative_time']=df_ben_day_2['Heure']-df_ben_day_2['Heure'].min()
df5_day_1['relative_time']=df5_day_1['Heure']-df5_day_1['Heure'].min()
df6_day_1['relative_time']=df6_day_1['Heure']-df6_day_1['Heure'].min()

<ipython-input-20-6c75816b8b57>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_day_1['relative_time']=df5_day_1['Heure']-df5_day_1['Heure'].min()
<ipython-input-20-6c75816b8b57>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6_day_1['relative_time']=df6_day_1['Heure']-df6_day_1['Heure'].min()


In [22]:
df_ben_day_2['relative_time'].sort_values()

,relative_time
2649,0 days 00:00:00
2690,0 days 00:00:00
2653,0 days 00:00:01
2654,0 days 00:00:01
2655,0 days 00:00:01
...,...
36448,0 days 18:14:58
25771,0 days 18:14:58
29449,0 days 18:14:58
20063,0 days 18:14:59


In [25]:
df6_day_1[df6_day_1['Dst IP'].isin(L_5_1)]['relative_time'].sort_values()

,relative_time
0,0 days 00:00:01
1,0 days 00:00:01
464877,0 days 00:00:01
454147,0 days 00:00:02
479643,0 days 00:00:03
...,...
458479,0 days 03:39:58
898770,0 days 03:39:59
477816,0 days 03:39:59
456803,0 days 03:39:59


In [21]:
data=pd.concat([df_ben_day_2,df5_day_1[df5_day_1['Dst IP'].isin(L_5_1)],df6_day_1[df6_day_1['Dst IP'].isin(L_6_1)]],axis=0)

In [23]:
data['relative_time'].sort_values()

,relative_time
0,0 days 00:00:00
343151,0 days 00:00:00
1,0 days 00:00:00
2690,0 days 00:00:00
2649,0 days 00:00:00
...,...
36448,0 days 18:14:58
22811,0 days 18:14:58
29449,0 days 18:14:58
20063,0 days 18:14:59


Time-aware Tensor graph

In [36]:
def create_graph_tensor(df, window_size=60):
    # Extract unique source and destination IPs
    src_ips = df['Src IP'].unique()
    dst_ips = df['Dst IP'].unique()

    # Create mappings for src_ip and dst_ip to indices
    src_ip_to_idx = {ip: idx for idx, ip in enumerate(src_ips)}
    dst_ip_to_idx = {ip: idx for idx, ip in enumerate(dst_ips)}

    # Create time windows
    df['time_window'] = df['relative_time'].astype(np.int64) // (10**9 * window_size)
    time_windows = sorted(df['time_window'].unique())

    # Create adjacency tensor [time_windows, Src IP, Dst IP, features]
    tensor = np.zeros((len(time_windows), len(src_ips), len(dst_ips), 4))  # 4 features (count, bytes, duration, packets)

    # Group by time window, source and destination
    for _, group in df.groupby(['time_window', 'Src IP', 'Dst IP']):
        t_idx = np.where(time_windows == group['time_window'].iloc[0])[0][0]
        src_idx = src_ip_to_idx[group['Src IP'].iloc[0]]
        dst_idx = dst_ip_to_idx[group['Dst IP'].iloc[0]]

        # Aggregated features
        tensor[t_idx, src_idx, dst_idx, 0] = len(group)  # count
        tensor[t_idx, src_idx, dst_idx, 1] = group['Flow Bytes/s'].sum()  # bytes
        tensor[t_idx, src_idx, dst_idx, 2] = group['Flow Duration'].mean()  # duration
        tensor[t_idx, src_idx, dst_idx, 3] = group['Flow Packets/s'].sum()  # packets

    return tensor, src_ip_to_idx, dst_ip_to_idx, time_windows


In [37]:
tensor, src_ip_to_idx, dst_ip_to_idx, time_windows=create_graph_tensor(data)

In [38]:
tensor.size

2930283136

In [39]:
tensor.shape

(1096, 484, 1381, 4)

In [40]:
np.count_nonzero(tensor)

483071

Time-binned Tensor graph with adaptive resolution

In [ ]:
from datetime import timedelta

def create_adaptive_time_bins(df, min_bin_size=60, max_bin_size=300, activity_threshold=20):
    # The 'relative_time' column is already a timedelta, no need to convert
    #timestamps = pd.to_datetime(df['relative_time']).sort_values().tolist()
    timestamps = df['relative_time'].sort_values().tolist()

    bins = []
    current_bin_start = timestamps[0]
    current_events = []

    for ts in timestamps:
        current_events.append(ts)

        elapsed_time = (ts - current_bin_start).total_seconds()

        # Vérifie si on doit créer un bin
        if len(current_events) >= activity_threshold or elapsed_time >= max_bin_size:
            if len(current_events) >= activity_threshold:
                # Forte activité ➜ plus petit bin
                avg_spacing = elapsed_time / len(current_events)
                bin_size = min(max_bin_size, max(min_bin_size, avg_spacing * 5))
            else:
                # Faible activité ➜ grand bin
                bin_size = max_bin_size

            bin_end = current_bin_start + timedelta(seconds=bin_size)
            bins.append((current_bin_start, bin_end))

            # Réinitialiser pour le bin suivant
            current_bin_start = bin_end
            current_events = [e for e in current_events if e >= bin_end]

    # Traiter les éventuels événements restants
    if current_bin_start < timestamps[-1]:
        bins.append((current_bin_start, timestamps[-1]))

    return bins

In [ ]:
def create_graph_tensor_with_bins(df, bins):
    # 1. Obtenir tous les IPs uniques
    all_ips = set(df['Src IP'].unique()) | set(df['Dst IP'].unique())
    ip_to_idx = {ip: idx for idx, ip in enumerate(sorted(all_ips))}

    n_devices = len(all_ips)
    n_features = 4  # count, bytes, duration, packets
    n_bins = len(bins)

    # 2. Initialiser le tenseur [temps, src, dst, features]
    tensor = np.zeros((n_bins, n_devices, n_devices, n_features))

    # 3. Associer chaque ligne à un bin temporel
    df['bin_index'] = -1  # initialisation

    for i, (start, end) in enumerate(bins):
        mask = (df['relative_time'] >= start) & (df['relative_time'] < end)
        df.loc[mask, 'bin_index'] = i

    # 4. Groupement et remplissage du tenseur
    for _, group in df[df['bin_index'] != -1].groupby(['bin_index', 'Src IP', 'Dst IP']):
        bin_idx = group['bin_index'].iloc[0]
        src_idx = ip_to_idx[group['Src IP'].iloc[0]]
        dst_idx = ip_to_idx[group['Dst IP'].iloc[0]]

        tensor[bin_idx, src_idx, dst_idx, 0] = len(group)  # count
        tensor[bin_idx, src_idx, dst_idx, 1] = group['Flow Bytes/s'].sum()
        tensor[bin_idx, src_idx, dst_idx, 2] = group['Flow Duration'].mean()
        tensor[bin_idx, src_idx, dst_idx, 3] = group['Flow Packets/s'].sum()

    return tensor, ip_to_idx, bins


Some Methods overview tha I will explain
.. Tensor residuals for anomalies detection

-Low-rank tensor using a tensor decomposition method
-residuals calculation on the new dataset
-anomalies detection using : either a supervised learning or an unsupervised learning

..Nonnegative Tensor factorization
-
.. Tensor Robust PCA
..